Team member names: Hetsvi Navnitlal and Emily Kwan 

Advanced Geopandas: spatial join and geoprocessing

In [ ]:
%matplotlib inline 
import geopandas as gpd
import pandas as pd

# add packages as needed

import os
current_dir = os.getcwd()
print(current_dir)

this will be a function to return geometric characteristics and neighbors of any polygon in an input dataframe (input_gdf) with a given name (poly_name) in a specific column (poly_name_field).

In [ ]:
shpFileIn = '../CA_Counties_TIGER2016.shp'
sd_parks = gpd.read_file(shpFileIn)
sd_parks.info

In [ ]:
def poly_geometry(input_gdf, poly_name_field,poly_name):
    cols = ['NAME', 'ALAND', 'geometry']
    sub_regions = input_gdf[cols].dissolve(by='NAME', aggfunc = 'sum', as_index=False)
    #print("sub_region", sub_regions, "sub_region")
    crs = sub_regions.crs
    #print(crs)
    sd = sub_regions.loc[sub_regions[poly_name_field] == poly_name]
    #print(sd)
    poly_coords = sd['geometry']
    #print("poly_coord", poly_coords, "poly_coord")
    area = sd.area/(2590000) #sd.ALAND*(0.0015625)
    #print(area)
    bounding_box = sd.geometry.bounds
    #print(bounding_box)
    #neighbors = sub_regions[sub_regions.touches(sd.geometry)].NAME.tolist()
    sub_regions["NEIGHBOURS"] = None
    for i, rows in sub_regions.iterrows():
        neighbors = sub_regions[~sub_regions.disjoint(rows.geometry)].NAME.tolist()
        neighbors = [j for j in neighbors if rows.NAME != j]
        sub_regions.at[i, "NEIGHBOURS"] = ", ".join(neighbors)
        
    neighours = sub_regions.loc[sub_regions["NAME"] == poly_name]["NEIGHBOURS"].tolist()
    neighours =neighours[0].split(", ")
    return crs, poly_coords, area, bounding_box, neighours 

In [ ]:
# running the function

input_gdf = gpd.read_file('../CA_Counties_TIGER2016.shp')
poly_name_field = 'NAME'  # or whatever is in the file
poly_name = 'San Diego' # or whatever is in the file

crs,poly_coords,area,bbox,neighbors = poly_geometry(input_gdf, poly_name_field, poly_name)

print(crs)
print(poly_coords)
print(area)
print(bbox)
print(neighbors)

dataset description:
The title of this dataset is CA County Boundries. The URL for this is https://data.ca.gov/dataset/ca-geographic-boundaries/resource/091ff50d-bb24-4537-a974-2ce89c6e8663. This dataset has the county boundries for CA state from the USA census bureau's 2016 MAF/TIGER database. This data set has 58 records (0-57 index). The coordinate reference system is polygons which is {'init': 'epsg:3857'}. According to the metadata we can tell that it was compiled from the data thats US Census Bureau collectes as a part of the Master Address File. The unit consistency of the area of the data and combining similar areas into which fall under one name etc. are some of the data quality considerations which we had to take care of.

## food desert
Food choices in low and medium income (LMI) neighborhoods are limited, as these areas lack healthy food stores or other fresh food delivery mechanisms. There may be more fast food stores than fresh produce shops. Population is relatively poor, and filling the food basket with healthy options hasn't been an overwhelming priority for many households. Such areas are referred to as "food deserts".
See these articles:
https://www.sandiegouniontribune.com/news/data-watch/sdut-san-diego-food-deserts-2016may15-htmlstory.html (see food desert definitions here)
https://www.npr.org/sections/thesalt/2013/03/13/174112591/how-to-find-a-food-desert-near-you

We'll analyze the current state of food deserts, using lists of businesses that operate in San Diego, and social-demographic data from SanDAG (by census tracts).
For the list of businesses, let's use City of San Diego tax certificates: https://data.sandiego.gov/datasets/business-listings/. 
There is also "SanDAG's point locations of business sites" in San Diego County (see metadata at http://rdw.sandag.org/Account/GetFSFile.aspx?dir=Business&Name=BUSINESS_SITES.pdf). 
We'll also get relevant spatial data layers from SanDAG: Promise Zone, and census tract boundaries.

dataset descriptions:
promise: contains geo information for the San Diego Promise Zone that spans the city's most disadvantaded and underserved communites.
census: contains the polygons representing the 2010 US Census Bureau census tracts for San Diego county
juris: contains the different municipal jurisdictions for parcels, roads, addresses in San DIego.
business: contains locations of business sites in San Diego County
social: contains social-demographic data by census tracts.

In [ ]:
#1 Download the data 
promise = gpd.read_file('../Promise_Zone_SD.shp')
census = gpd.read_file('../CENSUS_TRACTS_2010.shp')
juris = gpd.read_file('../MUNICIPAL_BOUNDARIES.shp')
business = gpd.read_file('sd_businesses_active_since08_datasd.csv')
socio = gpd.read_file('socio-demographic.csv')
socio['TRACT'] = (socio['TRACT']).astype(float)
socio = socio.drop(['geometry'], axis = 1)
#print(type(socio['TRACT'][0]))
socio_census = census.merge(socio, how='outer', on='TRACT')
census
socio

In [ ]:
#2 Subset the businesses
business = business.loc[business['account_status'] == 'Active']
health_list = ['FOOD (HEALTH) SUPPLEMENT STORES', 'GROCERY STORES', 'GROCERY (EXCEPT CONVENIENCE) STORES']
unhealth_list = ['FOOD SERVICES & DRINKING PLACES', 'FULL-SERVICE RESTAURANTS', 'LIMITED-SERVICE EATING PLACES', 'LIMITED-SERVICE RESTAURANTS', 'BEER, WINE & LIQUOR STORES', 'TOBACCO STORES', 'CONVENIENCE STORES']
health = business[business['naics_description'].isin(health_list)]
unhealth = business[business['naics_description'].isin(unhealth_list)]

In [ ]:
#3 Determine census tracts intersecting with the promise zone, and all other census tracts within the boundary of San Diego.
cols = ['NAME', 'geometry']
sub_regions = juris[cols].dissolve(by='NAME', aggfunc = 'sum', as_index = False)
sd = sub_regions.loc[sub_regions['NAME'] == 'SAN DIEGO']
sd

In [ ]:
#3
polygons = []
polygon = pd.DataFrame()
for i, rows in census.iterrows():
    for j, ro in sd.iterrows():
    #print(i.geometry)
   # print(sd['geometry'])
        if ro['geometry'].contains(rows['geometry']):
            dict1 = {}
            dict1.update(rows)
            polygons.append(dict1)
            #polygons.append(ro['geometry'].intersects(rows['geometry']))
polygons  
polygon = pd.DataFrame(polygons)
polygon

lst = []
for k , polygon_city in polygon.iterrows():
    for l, po_city in promise.iterrows():
        if po_city['geometry'].intersects(polygon_city['geometry']):
            lst.append(po_city['geometry'].intersects(polygon_city['geometry']))
        else:
            lst.append(False)

polygon['PROMISE'] = lst
#print(lst)

In [ ]:
#4 calculate the number of healthy and not-healthy options within that tract
from shapely.geometry import Point
import numpy as np
health['longitude'] = health['longitude'].replace('', np.nan)
health = health.dropna(subset = ['longitude', 'latitude'])
health['geometry'] = health.apply(lambda x: Point(float(x['longitude']), float(x['latitude'])), axis = 1)
unhealth['longitude'] = unhealth['longitude'].replace('', np.nan)
unhealth = unhealth.dropna(subset = ['longitude', 'latitude'])
unhealth['geometry'] = unhealth.apply(lambda x: Point(float(x['longitude']), float(x['latitude'])), axis = 1)
health.crs = {'init' :'epsg:4326'}
health = health.to_crs(epsg=2230)
unhealth.crs = {'init' :'epsg:4326'}
unhealth = unhealth.to_crs(epsg=2230)
for i, tract in polygon.iterrows():
    num_health = 0
    num_unhealth = 0
    for j, food in health.iterrows():
        #print((tract.geometry))
        if (food.geometry).within(tract.geometry):
            #print(food.geometry)
            num_health+=1
    for x, xfood in unhealth.iterrows():
        if (xfood.geometry).within(tract.geometry):
            num_unhealth+=1
    polygon.loc[i, 'health'] = num_health
    polygon.loc[i, 'unhealth'] = num_unhealth
polygon['health_ratio'] = polygon['health']/(polygon['health']+polygon['unhealth'])
polygon['unhealth_ratio'] = polygon['unhealth']/(polygon['health']+polygon['unhealth'])
polygon

In [ ]:
#5 Compute the ratio of healthy and non-healthy options, and build a regression model showing how it depends on social-demographic characteristics, such as median income or number of people in lowest and highest income brackets
from sklearn import linear_model as lm
from sklearn.pipeline import Pipeline
import sklearn.preprocessing as pp
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

feats = ['ORDINAL', 'TRACT']
polygon['healthy'] = polygon['health'] > polygon['unhealth']
socio_poly = polygon.merge(socio, how='outer', on='TRACT')
socio_poly = socio_poly.dropna(subset=['health_ratio'])
prom = socio_poly[socio_poly['PROMISE'] == True ]
other = socio_poly[socio_poly['PROMISE'] == False ]

promX = prom[feats]
promY = prom['health_ratio']
otherX = other[feats]
otherY = other['health_ratio']

x_train, x_test, y_train, y_test = train_test_split(promX, promY, test_size=0.25)
cat_transformer = Pipeline(steps=[('onehot', pp.OneHotEncoder())])
preproc = ColumnTransformer(transformers=[('cat', cat_transformer, feats)])
lin_reg = lm.LinearRegression()
pl = Pipeline(steps=[('preprocessor', preproc), ('linear', lin_reg)])
pl.fit(x_train, y_train)
score = pl.score(x_test, y_test)
print("Regression score for tracts within the promise zone: {}".format(score))

x_train, x_test, y_train, y_test = train_test_split(otherX, otherY, test_size=0.25)
cat_transformer = Pipeline(steps=[('onehot', pp.OneHotEncoder())])
preproc = ColumnTransformer(transformers=[('cat', cat_transformer, feats)])
lin_reg = lm.LinearRegression()
pl = Pipeline(steps=[('preprocessor', preproc), ('linear', lin_reg)])
pl.fit(x_train, y_train)
score = pl.score(x_test, y_test)
print("Regression score for tracts outside the promise zone: {}".format(score))

#5 description The classifier score for applying a linear regression to tracts within the promise zone was 0.856366 while the score for applying a linear regression to tracts outside of the promise zone was 0.30966. This can indicate that for areas within the promise zone, income level and geographic location are closely correlated with the ratio of healthy restaurants that exist in the area while the same cannot be said for areas outside of the promise zone. However, we should take pause in coming to this conclusion due to the fact that we had very little applicable healthy and non-healthy restaurant data, resulting in many healthy ratios ending up being 0 due to not having found any healthy or unhealthy restaurants in the business dataset. This could lead to reduced reliability in the conclusions we can draw from this regression study.

In [ ]:
#6 map showing the ratio of healthy and not healthy food options, and the locations of stores offering healthy food
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
#import GeoDataFrame

polygon = GeoDataFrame(polygon, geometry = polygon.geometry)  #, crs=crs, geometry = polygon.geometry)
map_df = polygon
map_df.head()
polygon['unhealth_ratio'] = polygon['unhealth_ratio'].fillna(0)
#map_df.plot()

variable = 'unhealth_ratio'
vmin, vmax = 0, 1

fig, ax = plt.subplots(1, figsize = (10, 6))
a = map_df.plot(column = variable, cmap = 'BuGn', linewidth = 0.8, ax=ax, edgecolor = '0.8')
#fig, ax = plt.subplot(1, figsize = (10,6))
ax.axis('off')
ax.set_title('Ratio of the Unhealthy')
color_bar = plt.cm.ScalarMappable(cmap='BuGn', norm=plt.Normalize(vmin=vmin, vmax=vmax))
color_bar._A = []
#health.plot(ax = a, marker='*', color='black', markersize = 2)
#health.plot(ax = a, marker='*', color='black', markersize = 10)
b = health.plot(ax = a, marker = 'o', color='black', markersize = 10)
#print(health)
#health.plot(ax = a, marker = 'o', color = 'k', markersize = 10)
col_bar = fig.colorbar(color_bar) 
promise.plot(ax = b, color = 'None', edgecolor = 'b')
#promise.plot(ax = b, linewidth = 0.15)
#promise.plot(ax = b, linewidth = 0.5)
#promise.plot(ax = b, color = 'None', edgecolor = 'black')
#promise.plot(ax = b, color = 'None', edgecolor = 'c')
#promise.plot(ax = b, color = 'None', edgecolor = 'black', linewidth = 0.15)
#promise.plot(ax = b, color = 'None', edgecolor = 'blue', linewidth = 0.10)

